In [2]:
from os import walk
from os.path import join
import pandas as pd

# for root, dirs, files in walk("/Users/nailbiter/Documents/duolingo/"):
#     for name in files:
#         print(join(root,name))
decks = pd.read_csv("/Users/nailbiter/Documents/duolingo/decks.csv")
decks

,name,description,coverImage,cards,privacy,language,deleted,createdAt,updatedAt
0,N2 tango 2500 (9),NaN,NaN,### Card 1\nFront\n* しかも\n\nBack\n* moreover\n...,Everyone,en,False,2020-05-08 05:50:53,2020-05-08 05:50:53
1,N2 tango 2500 (8),NaN,NaN,"### Card 1\nFront\n* つらい\n\nBack\n* painful, d...",Everyone,en,False,2020-05-07 23:37:28,2020-05-08 05:36:44
2,N2 tango 2500 (8),NaN,NaN,### Card 1\nFront\n* 目撃\n\nBack\n* witness\n* ...,Everyone,en,False,2020-05-06 13:33:01,2020-05-07 13:50:35
3,N2 tango 2500 (7),NaN,NaN,### Card 1\nFront\n* 定期的な\n\nBack\n* periodic\...,Everyone,en,False,2020-05-06 08:18:14,2020-05-06 13:24:44
4,N2 tango 2500 (6),NaN,NaN,### Card 1\nFront\n* 本格的な\n\nBack\n* serious\n...,Everyone,en,False,2020-05-06 06:40:35,2020-05-06 08:14:25
5,N2 tango 2500 (5),NaN,NaN,### Card 1\nFront\n* 組む\n\nBack\n* to join\n* ...,Everyone,en,False,2020-04-28 14:42:32,2020-05-06 05:53:02
6,N2 tango 2500 (4),NaN,NaN,### Card 1\nFront\n* 高等な\n\nBack\n* superior\n...,Everyone,en,False,2020-04-26 07:11:33,2020-04-28 14:36:16
7,N2 tango 2500 (3),NaN,NaN,### Card 1\nFront\n* 可燃ゴミ\n\nBack\n* burnable ...,Everyone,en,False,2020-04-13 13:04:52,2020-05-14 14:07:45
8,N2 tango 2500 (2),NaN,NaN,### Card 1\nFront\n* 残高\n\nBack\n* balance\n* ...,Only me,en,False,2020-03-27 01:58:32,2020-05-10 03:51:27
9,N2 tango 2500 (1),N2 words 2500,NaN,### Card 1\nFront\n* 一家\n\nBack\n* family\n* い...,Everyone,en,False,2020-03-14 16:06:01,2020-03-27 01:57:37


In [44]:
from functools import reduce
from re import match

class Card:
    def __init__(self,lines):
        self._lines = lines
        front_i, back_i = lines.index("Front"),lines.index("Back")
        self._front = [s[2:] for s in lines[front_i+1:back_i]]
        self._back = [s[2:
                       ] for s in lines[back_i+1:]]
    def __str__(self):
        _d = {"front":self._front,"back":self._back}
        return f"Card({_d})"
    def to_dict(self):
        return {"front":self._front,"back":self._back}
class Deck:
    def __init__(self,s,name):
        lines = s.split("\n")
        def _reducer(val,elt):
            if match(r"### Card \d+",elt) is not None:
                val["res"].append(val["buf"])
                val["buf"] = []
            else:
                val["buf"].append(elt)
            return val
        _deck = reduce(_reducer, lines, {"buf":[],"res":[]})
        if _deck["buf"]:
            _deck["res"].append(_deck["buf"])
        _deck["res"] = [Card([aa for aa in a if aa]) for a in _deck["res"] if a]
        self._deck = _deck["res"]
        self._name = name
    def get_name(self):
        return self._name
    def get_size(self):
        return len(self._deck)
    def __str__(self):
        _s = ", ".join(str(c) for c in self._deck)
        return f"Deck(name={self._name},[{_s}])"
    def to_dict(self):
        return {self._name:[c.to_dict() for c in self._deck]}
        

decks_df = pd.DataFrame({"cards":[Deck(s=cards,name=name) for cards,name in zip(decks["cards"],decks["name"])]})

In [51]:
# from pymongo import MongoClient
# from tqdm import tqdm

# mongo_client = MongoClient()
# coll = mongo_client.alex_flashcards.cards

# _deck = [d.to_dict() for d in decks_df["cards"]]
# for d in tqdm(_deck):
#     k,v = list(d.items())[0]
#     for c in v:
#         doc = {"tags":[k,"tinycards"],**c}
#         assert len(doc["front"])==1
#         doc["front"] = doc["front"][0]
#         coll.insert_one(doc)

100%|██████████| 56/56 [00:01<00:00, 55.75it/s]


In [43]:
_df = decks_df[["N2 tango" in d.get_name() for d in decks_df["cards"]]].copy()
_df["name"] = [d.get_name() for d in _df["cards"]]
_df["size"] = [d.get_size() for d in _df["cards"]]
num_of_cards = sum(_df["size"])
print(f"num_of_cards: {num_of_cards}")
# _df

num_of_cards: 1364


In [40]:
_df = schedule_df.groupby("book").sum()
print(_df)
sum(_df["words"])

               page_count  words
book                            
None                    0     38
n2 in 20 days         165    476
n2 listening          102    289
n2 reading            196    561


1364

In [42]:
from datetime import datetime, timedelta
from math import ceil
from copy import deepcopy
# from math impo

PAGE_COUNTS = {
    "n2 listening":102,
    "n2 reading":196,
    "n2 in 20 days":165,
}
MOCK_EXAMS = 1
TEST_DATE = datetime(2020,12,6)

start_date = datetime.today()+timedelta(days=2)
days_remain = (TEST_DATE-start_date).days
average_pages_per_day = ceil(sum(PAGE_COUNTS.values())/days_remain)
average_words_per_day = ceil(num_of_cards/days_remain)

records = []
_noc = num_of_cards
_ski = 0
_pc = deepcopy(PAGE_COUNTS)
_keys = [*list(_pc),"",""]
while sum(_pc.values())>0 or _noc>0:
    if _pc.get(_keys[_ski],0)==0:
        _ski += 1
    if _ski<len(_pc):
        norm = min(_pc.get(list(_pc)[_ski],0),average_pages_per_day)
        _pc[list(_pc)[_ski]] -= norm
    else:
        norm = 0
    norm_words = min(average_words_per_day,_noc)
    _noc -= norm_words
    records.append({"book":list(_pc)[_ski] if _ski<len(_pc) else "None", "page_count":norm,"date":start_date.date(),"words":norm_words})
    start_date += timedelta(days=1)

schedule_df = pd.DataFrame(records)
schedule_df

,book,page_count,date,words
0,n2 listening,6,2020-09-12,17
1,n2 listening,6,2020-09-13,17
2,n2 listening,6,2020-09-14,17
3,n2 listening,6,2020-09-15,17
4,n2 listening,6,2020-09-16,17
...,...,...,...,...
76,n2 in 20 days,6,2020-11-27,17
77,n2 in 20 days,3,2020-11-28,17
78,None,0,2020-11-29,17
79,None,0,2020-11-30,17
